In [6]:
import json
import random
from datetime import datetime, timedelta, timezone
import requests
# Constants defining schema constraints
NUM_PATIENTS = 2
PATIENT_ID_FORMAT = "P{:04d}"  # Patient IDs: P0001 to P0050
AGE_RANGE = (60, 90)  # Age between 60 and 90 inclusive
DATE_RANGE_DAYS = 2  # Generate data for past 60 days
OBSERVATION_WINDOWS = [
    (0, 6),
    (6, 12),
    (12, 18),
    (18, 24),
]  # Observation time windows in hours (UTC)

# Health states and their generation probabilities
STATES = ["normal", "uncomfortable", "sick"]
STATE_PROBS = [0.7, 0.2, 0.1]  # Normal is most frequent

def generate_iso8601_utc(day: datetime.date, start_hour: int, end_hour: int):
    """
    Generate ISO 8601 timestamps for observationStart and observationEnd,
    within the given observation window on the specified day.
    """
    start_dt = datetime.combine(day, datetime.min.time(), tzinfo=timezone.utc) + timedelta(hours=start_hour)
    end_dt = datetime.combine(day, datetime.min.time(), tzinfo=timezone.utc) + timedelta(hours=end_hour)
    # Randomize start within the window, leave room for at least 30 mins observation
    max_start_minute = (end_dt - start_dt).seconds // 60 - 30
    random_start = start_dt + timedelta(minutes=random.randint(0, max_start_minute))
    # Observation duration between 30 mins and 90 mins
    duration_minutes = random.randint(30, 90)
    random_end = random_start + timedelta(minutes=duration_minutes)
    if random_end > end_dt:
        random_end = end_dt
    # Return ISO 8601 with 'Z' for UTC suffix
    return random_start.isoformat().replace("+00:00", "Z"), random_end.isoformat().replace("+00:00", "Z")

def generate_nursingNote(state: str) -> str:
    """Generate nursingNote field text depending on patient state."""
    notes = {
        "normal": [
            "Patient is stable with no complaints.",
            "Vitals within normal limits.",
            "Patient resting comfortably."
        ],
        "uncomfortable": [
            "Patient reported mild chest pain after breakfast.",
            "Blood pressure slightly elevated.",
            "Patient feeling tired and weak."
        ],
        "sick": [
            "Patient shows signs of distress and discomfort.",
            "Elevated heart rate and temperature noted.",
            "Patient refused physiotherapy session."
        ],
    }
    return random.choice(notes[state])

def generate_medications(obs_start_iso: str, state: str):
    """Generate medications array according to schema."""
    med1_compliance = {
        "normal": "Taken",
        "uncomfortable": random.choices(["Taken", "Delayed"], [0.7, 0.3])[0],
        "sick": random.choices(["Missed", "Delayed", "Taken"], [0.5, 0.3, 0.2])[0],
    }
    med1 = {
        "name": "Aspirin",
        "dose": "75mg",
        "scheduledTime": obs_start_iso,
        "complianceStatus": med1_compliance[state],
    }
    obs_start_dt = datetime.fromisoformat(obs_start_iso.replace("Z", "+00:00"))
    med2_time = (obs_start_dt + timedelta(hours=12)).isoformat().replace("+00:00", "Z")
    med2 = {
        "name": "Metformin",
        "dose": "500mg",
        "scheduledTime": med2_time,
        "complianceStatus": "Taken",
    }
    return [med1, med2]

def generate_vitals(state: str):
    """Generate vitals object as per schema with realistic values."""
    if state == "normal":
        hr = random.randint(65, 80)
        spo2 = random.randint(95, 99)
        temp = round(random.uniform(36.4, 37.0), 1)
        sys = random.randint(110, 130)
        dia = random.randint(70, 80)
    elif state == "uncomfortable":
        hr = random.randint(75, 90)
        spo2 = random.randint(92, 97)
        temp = round(random.uniform(37.0, 37.5), 1)
        sys = random.randint(130, 140)
        dia = random.randint(80, 85)
    else:  # sick
        hr = random.randint(90, 110)
        spo2 = random.randint(88, 93)
        temp = round(random.uniform(37.6, 38.5), 1)
        sys = random.randint(140, 150)
        dia = random.randint(85, 95)

    return {
        "heartRate": {"value": hr, "unit": "bpm"},
        "spo2": {"value": spo2, "unit": "%"},
        "temperature": {"value": temp, "unit": "°C"},
        "bloodPressure": {"systolic": sys, "diastolic": dia, "unit": "mmHg"},
    }

def generate_adls(state: str):
    """Generate ADLs object with exactly these 6 keys as per schema."""
    if state == "normal":
        return {
            "stepsTaken": random.randint(2000, 5000),
            "calorieIntake": random.randint(1800, 2500),
            "sleepHours": round(random.uniform(6, 8), 1),
            "waterIntakeMl": random.randint(1500, 2000),
            "mealsSkipped": 0,
            "exerciseMinutes": random.randint(10, 45),
        }
    elif state == "uncomfortable":
        return {
            "stepsTaken": random.randint(1000, 3000),
            "calorieIntake": random.randint(1200, 1800),
            "sleepHours": round(random.uniform(4, 6), 1),
            "waterIntakeMl": random.randint(1000, 1500),
            "mealsSkipped": random.choices([0, 1], [0.7, 0.3])[0],
            "exerciseMinutes": random.randint(0, 20),
        }
    else:  # sick
        return {
            "stepsTaken": random.randint(0, 1000),
            "calorieIntake": random.randint(500, 1200),
            "sleepHours": round(random.uniform(3, 5), 1),
            "waterIntakeMl": random.randint(500, 1200),
            "mealsSkipped": random.choices([1, 2], [0.7, 0.3])[0],
            "exerciseMinutes": 0,
        }

def generate_behaviourTags(state: str):
    """Generate behaviourTags array as per schema."""
    tags_map = {
        "normal": [],
        "uncomfortable": ["poorDiet", "refusedActivity"],
        "sick": ["poorDiet", "refusedActivity", "noncompliantMedication"],
    }
    return tags_map[state]

def generate_emotionAnalysis(state: str):
    """Generate emotionAnalysis object with 'tags' and 'confidenceScores'."""
    if state == "normal":
        return {"tags": [], "confidenceScores": {}}
    elif state == "uncomfortable":
        return {
            "tags": ["frustrated"],
            "confidenceScores": {"frustrated": round(random.uniform(0.6, 0.85), 2)},
        }
    else:  # sick
        return {
            "tags": ["frustrated", "lowEnergy"],
            "confidenceScores": {
                "frustrated": round(random.uniform(0.75, 0.95), 2),
                "lowEnergy": round(random.uniform(0.7, 0.9), 2),
            },
        }

def generate_clinicalSummary(state: str):
    """Generate clinicalSummary string."""
    summaries = {
        "normal": "Patient is stable and comfortable.",
        "uncomfortable": "Patient shows mild discomfort and irregular vitals.",
        "sick": "Patient condition is poor with significant symptoms and low compliance.",
    }
    return summaries[state]

def generate_entitiesExtracted():
    """Generate empty entitiesExtracted dictionary with specified keys."""
    return {"symptoms": [], "vitals": [], "procedures": [], "medications": []}

def generate_baselineStats():
    """Generate baselineStats dictionary with all fields set to null."""
    return {
        "avgHeartRate": None,
        "avgSpo2": None,
        "avgSleepHours": None,
        "avgCalorieIntake": None,
        "avgScreenTimeMinutes": None,
        "avgBathroomVisits": None,
        "usualDietCompliance": None,
    }

def generate_alerts():
    """Generate empty alerts list."""
    return []

def generate_patient_observations():
    """Generate synthetic patient observation data strictly matching the schema."""
    # Use timezone-aware datetime for start_date to avoid deprecation warning
    start_date = datetime.now(timezone.utc).date() - timedelta(days=DATE_RANGE_DAYS)
    patient_ids = [PATIENT_ID_FORMAT.format(i + 1) for i in range(NUM_PATIENTS)]

    all_records = []

    for patient_id in patient_ids:
        age = random.randint(*AGE_RANGE)
        gender = random.choice(["Male", "Female"])

        for day_offset in range(DATE_RANGE_DAYS):
            current_day = start_date + timedelta(days=day_offset)

            for (window_start, window_end) in OBSERVATION_WINDOWS:
                state = random.choices(STATES, STATE_PROBS)[0]

                observationStart, observationEnd = generate_iso8601_utc(current_day, window_start, window_end)

                nursingNote = generate_nursingNote(state)
                medications = generate_medications(observationStart, state)
                vitals = generate_vitals(state)
                adls = generate_adls(state)
                behaviourTags = generate_behaviourTags(state)
                emotionAnalysis = generate_emotionAnalysis(state)
                clinicalSummary = generate_clinicalSummary(state)
                entitiesExtracted = generate_entitiesExtracted()
                baselineStats = generate_baselineStats()
                alerts = generate_alerts()

                record = {
                    "patientId": patient_id,
                    "age": age,
                    "gender": gender,
                    "observationStart": observationStart,
                    "observationEnd": observationEnd,
                    "nursingNote": nursingNote,
                    "medications": medications,
                    "vitals": vitals,
                    "adls": adls,
                    "behaviourTags": behaviourTags,
                    "emotionAnalysis": emotionAnalysis,
                    "clinicalSummary": clinicalSummary,
                    "entitiesExtracted": entitiesExtracted,
                    "baselineStats": baselineStats,
                    "alerts": alerts,
                }

                all_records.append(record)

    return all_records

def post_data_to_google_sheet(api_url, data, batch_size=100):
    """
    Posts the generated data in batches to the Google Apps Script POST API.
    Batching avoids timeouts for large data.
    """
    headers = {"Content-Type": "application/json"}

    total = len(data)
    print(f"Total records to post: {total}")

    for i in range(0, total, batch_size):
        batch = data[i : i + batch_size]
        print(f"Posting batch {i // batch_size + 1} ({len(batch)} records)...")

        try:
            response = requests.post(api_url, data=json.dumps(batch), headers=headers)
            response.raise_for_status()
            resp_json = response.json()
            if resp_json.get("status") != "success":
                print(f"API error: {resp_json}")
                break
        except Exception as e:
            print(f"Failed to post batch: {e}")
            break

    print("Posting completed.")


if __name__ == "__main__":
    # 1. Generate synthetic data
    data = generate_patient_observations()

    # 2. Your Google Apps Script Web App POST URL
    GOOGLE_SHEET_API_URL = "https://script.google.com/macros/s/AKfycbyPOXMMnMb4kC5lbGEnqirRph_HcsLFgyfqWuDnNDmqEq9QDHnLKWNvN7_MvPks2A-v-w/exec"

    # 3. Post generated data in batches (adjust batch_size if needed)
    post_data_to_google_sheet(GOOGLE_SHEET_API_URL, data, batch_size=50)

Total records to post: 16
Posting batch 1 (16 records)...
Posting completed.
